# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
import nltk
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from io import StringIO

/home/vboxuser/dev/bertnews/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Загружаем отобранные статьи 

In [2]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(StringIO(read_data), orient='records')

Формируем предложения из статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложени и их длину

In [3]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"sentence": [], "len": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [s, len(s)]
print("Number of sentences: ", len(sentences.index))
sentences.head(3)

Number of sentences:  222


,sentence,len
0,В Госдуме предложили сделать старый Новый год ...,84
1,Председатель союза дачников Подмосковья и депу...,234
2,"""После длинных новогодних праздников людям тяж...",176


Формируем набор для обучения и тестовый набор

In [4]:
train, test = train_test_split(sentences, test_size=0.2, random_state=42)
train_text = train['sentence'].astype('str')
test_text = test['sentence'].astype('str')
train_labels = train.index
test_labels = test.index


Задание всех seed

In [5]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [14]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values.tolist(),
    padding = True,
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values.tolist(),
    padding = True,
    truncation = True
)

In [16]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

https://www.javatpoint.com/accuracy_score-in-sklearn

In [19]:
def compute_accuracy(Y_true, Y_pred):  
    correctly_predicted = 0  
    # iterating over every label and checking it with the true sample  
    for true_label, predicted in zip(Y_true, Y_pred):  
        if true_label == predicted:  
            correctly_predicted += 1  
    # computing the accuracy score  
    accuracy_score = correctly_predicted / len(Y_true)  
    return accuracy_score  

# Training the model using the Support Vector Classification class of sklearn  
svc = SVC()  
svc.fit(tokens_train['input_ids'], train_labels)  
  
# Computing the accuracy score of the model  
Y_pred = svc.predict(tokens_train['input_ids'])  
score = compute_accuracy(test_labels, Y_pred)  
print(score)  

0.0
